In [52]:
import requests
import pandas as pd
from csv import writer
import json
import time
import glob
import os
from pathlib import Path
import tqdm
from twarc import Twarc
from nltk.tokenize import TweetTokenizer
import numpy as np
from random import sample
from collections import defaultdict,OrderedDict
from tqdm import tqdm
from os import listdir, makedirs, remove
from os.path import isfile, join, exists

tk = TweetTokenizer()
tw = Twarc('asotxsa5cKuAeWjdYhZpABmN6', \
           'yhmMnNyE5bMx707yTcuS70NEoyFjvaPJnUCki1wHJG18XzLKYl', \
           '1116108383051046913-uzhQgRjVNAakb6pYKYUTCvW09donFA', \
           'XrHro3QvkuE0ivYXvQj9jufLDqk1EBNTcBnelXQ7hoM6P')
path = str(Path.cwd()) + '/data/'

In [4]:
fraction_we_need_to_sample = np.load("fraction_we_need_to_sample.npy", allow_pickle=True).tolist()

In [5]:
months = ['ukraine-russia-master/2022-02', 'ukraine-russia-master/2022-03', 'ukraine-russia-master/2022-04',
         'ukraine-russia-master/2022-05', 'ukraine-russia-master/2022-06', 'ukraine-russia-master/2022-07',
         'ukraine-russia-master/2022-08', 'ukraine-russia-master/2022-09', 'ukraine-russia-master/2022-10']

In [7]:
tweet_ids_per_day = defaultdict(list)
for month in tqdm(months):
    path = month
    month_num = month[22:]
    files = [f for f in listdir(path) if (isfile(join(path, f)))]
    for file in files:
        with open(join(path, file), 'r') as fp:
            for line in fp:
                date = file[20:-7]
                tweet_ids_per_day[date].append(line)
    tweet_ids_per_day_ordered = OrderedDict(sorted(tweet_ids_per_day.items()))
    
    tweet_ids_per_day_sampled = defaultdict(list)
    for day, tweet_ids in (tweet_ids_per_day_ordered.items()):
        tweet_ids_per_day_sampled[day] = sample(tweet_ids,fraction_we_need_to_sample[day])   
    
    np.save("sampled_tweet_ids/tweet_ids_per_day_sampled_"+str(month_num)+".npy", tweet_ids_per_day_sampled)
    del tweet_ids_per_day_ordered
    del tweet_ids_per_day
    del tweet_ids_per_day_sampled
    tweet_ids_per_day = defaultdict(list)

100%|█████████████████████████████████████████████| 9/9 [02:40<00:00, 17.83s/it]


In [ ]:
path = 'sampled_tweet_ids'
files = [f for f in listdir(path) if (isfile(join(path, f)))]
files.sort()
for file in tqdm(files):
    tweet_ids_month = np.load(join(path, file), allow_pickle=True).tolist()
    for day, tweet_ids in tweet_ids_month.items():
        output = file[26:-4] + "-" + day[3:] + ".txt"
        with open(os.path.join(path, output), 'w') as f:
            for tweet_id in tweet_ids:
                f.write(f"{tweet_id}\n")

In [39]:
import csv
def hydrate(path, file):
    list_of_tweets = []
    for tweet in tw.hydrate(open(path)):
        list_of_tweets.append(tweet)
    np.save(file, list_of_tweets)

In [40]:
import glob

In [41]:
files = glob.glob('sampled_tweet_ids/*[0-9].txt')

In [48]:
files.sort()

In [49]:
files

['sampled_tweet_ids/2022-02-22.txt',
 'sampled_tweet_ids/2022-02-23.txt',
 'sampled_tweet_ids/2022-02-24.txt',
 'sampled_tweet_ids/2022-02-25.txt',
 'sampled_tweet_ids/2022-02-26.txt',
 'sampled_tweet_ids/2022-02-27.txt',
 'sampled_tweet_ids/2022-02-28.txt',
 'sampled_tweet_ids/2022-03-01.txt',
 'sampled_tweet_ids/2022-03-02.txt',
 'sampled_tweet_ids/2022-03-03.txt',
 'sampled_tweet_ids/2022-03-04.txt',
 'sampled_tweet_ids/2022-03-05.txt',
 'sampled_tweet_ids/2022-03-06.txt',
 'sampled_tweet_ids/2022-03-07.txt',
 'sampled_tweet_ids/2022-03-08.txt',
 'sampled_tweet_ids/2022-03-09.txt',
 'sampled_tweet_ids/2022-03-10.txt',
 'sampled_tweet_ids/2022-03-11.txt',
 'sampled_tweet_ids/2022-03-12.txt',
 'sampled_tweet_ids/2022-03-13.txt',
 'sampled_tweet_ids/2022-03-14.txt',
 'sampled_tweet_ids/2022-03-15.txt',
 'sampled_tweet_ids/2022-03-16.txt',
 'sampled_tweet_ids/2022-03-17.txt',
 'sampled_tweet_ids/2022-03-18.txt',
 'sampled_tweet_ids/2022-03-19.txt',
 'sampled_tweet_ids/2022-03-20.txt',
 

In [61]:
path = 'sampled_tweet_ids'
for file in tqdm(files[63+62+34:]):
    outputfile = file.split('/')[1][:-4]
    if exists("sampled_tweets/" + outputfile):
        continue
    hydrate(file, "sampled_tweets/" + outputfile)

 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 47/63 [7:06:41<3:16:17, 736.07s/it]WARNING:twarc:rate limit exceeded: sleeping 480.4208948612213 secs


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [10:26:46<00:00, 596.94s/it]


In [43]:
tweets = np.load('sampled_tweets/2022-09-19.npy', allow_pickle=True).tolist()

In [45]:
len(tweets)

21671

In [58]:
30 +33

63